# Temporal CNN Embeddings

This notebook demonstrates the 1D dilated CNN for fast feature extraction from time series.

The temporal CNN uses dilated convolutions to capture patterns at multiple time scales, making it useful for time series classification, clustering, and similarity search.

## Installation

```bash
pip install ts2net[cnn]
# or
pip install ts2net torch
```


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

try:
    from ts2net.temporal_cnn import temporal_cnn_embeddings
    HAS_TORCH = True
except ImportError:
    HAS_TORCH = False
    print("PyTorch not installed. Install with: pip install torch")

np.random.seed(42)


## Example 1: Basic CNN Embeddings

Extract embeddings from a simple sine wave with noise.


In [ ]:
if not HAS_TORCH:
    print("Skipping example: PyTorch not installed")
else:
    # Create a simple time series
    t = np.linspace(0, 4*np.pi, 500)
    x = np.sin(t) + 0.1 * np.random.randn(500)
    
    # Extract embeddings
    embeddings = temporal_cnn_embeddings(
        x,
        window=50,
        stride=10,
        seed=42
    )
    
    print(f"Input: {len(x)} points")
    print(f"Output: {embeddings.shape[0]} windows, {embeddings.shape[1]} features per window")
    print(f"All finite: {np.all(np.isfinite(embeddings))}")
    
    # Visualize embeddings
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # Original time series
    axes[0].plot(x, linewidth=1, alpha=0.7)
    axes[0].set_title('Original Time Series', fontsize=12)
    axes[0].set_xlabel('Time')
    axes[0].set_ylabel('Value')
    axes[0].grid(True, alpha=0.3)
    
    # Embeddings heatmap
    im = axes[1].imshow(embeddings.T, aspect='auto', cmap='viridis', interpolation='nearest')
    axes[1].set_title('CNN Embeddings (64 features × windows)', fontsize=12)
    axes[1].set_xlabel('Window Index')
    axes[1].set_ylabel('Feature Index')
    plt.colorbar(im, ax=axes[1])
    
    plt.tight_layout()
    plt.show()


## Example 2: Multivariate Time Series

The CNN can handle multivariate time series by treating each feature as a channel.


In [ ]:
if not HAS_TORCH:
    print("Skipping example: PyTorch not installed")
else:
    # Create multivariate time series (3 features)
    t = np.linspace(0, 4*np.pi, 400)
    x = np.column_stack([
        np.sin(t),
        np.cos(t),
        np.sin(2*t),
    ])
    
    # Extract embeddings
    embeddings = temporal_cnn_embeddings(
        x,
        window=40,
        stride=8,
        seed=42
    )
    
    print(f"Input: {x.shape[0]} points, {x.shape[1]} features")
    print(f"Output: {embeddings.shape[0]} windows, {embeddings.shape[1]} features per window")
    
    # Visualize
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # Original multivariate series
    for i in range(x.shape[1]):
        axes[0].plot(x[:, i], label=f'Feature {i+1}', linewidth=1.5, alpha=0.7)
    axes[0].set_title('Multivariate Time Series', fontsize=12)
    axes[0].set_xlabel('Time')
    axes[0].set_ylabel('Value')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Embeddings
    im = axes[1].imshow(embeddings.T, aspect='auto', cmap='plasma', interpolation='nearest')
    axes[1].set_title('CNN Embeddings from Multivariate Input', fontsize=12)
    axes[1].set_xlabel('Window Index')
    axes[1].set_ylabel('Feature Index')
    plt.colorbar(im, ax=axes[1])
    
    plt.tight_layout()
    plt.show()


## Example 3: Custom Architecture

You can customize the CNN architecture by specifying channels, kernel sizes, and dilations.


In [ ]:
if not HAS_TORCH:
    print("Skipping example: PyTorch not installed")
else:
    x = np.sin(np.linspace(0, 4*np.pi, 300)) + 0.1 * np.random.randn(300)
    
    # Custom architecture: smaller channels, different dilations
    embeddings = temporal_cnn_embeddings(
        x,
        window=30,
        stride=5,
        channels=(16, 32, 48),  # Smaller channels
        kernel_size=3,           # Smaller kernel
        dilations=(1, 2, 4),     # Same dilations
        seed=42
    )
    
    print(f"Custom architecture: {embeddings.shape[1]} features (vs 64 default)")
    print(f"Created {embeddings.shape[0]} windows")
    
    # Compare with default
    embeddings_default = temporal_cnn_embeddings(
        x,
        window=30,
        stride=5,
        seed=42
    )
    
    print(f"Default architecture: {embeddings_default.shape[1]} features")
    
    # Visualize comparison
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    im1 = axes[0].imshow(embeddings.T, aspect='auto', cmap='viridis', interpolation='nearest')
    axes[0].set_title(f'Custom Architecture ({embeddings.shape[1]} features)', fontsize=12)
    axes[0].set_ylabel('Feature Index')
    plt.colorbar(im1, ax=axes[0])
    
    im2 = axes[1].imshow(embeddings_default.T, aspect='auto', cmap='viridis', interpolation='nearest')
    axes[1].set_title(f'Default Architecture ({embeddings_default.shape[1]} features)', fontsize=12)
    axes[1].set_xlabel('Window Index')
    axes[1].set_ylabel('Feature Index')
    plt.colorbar(im2, ax=axes[1])
    
    plt.tight_layout()
    plt.show()
